In [8]:
import pandas as pd
import numpy as np

star_df = pd.read_csv('celeb_name_list.csv')        # 연예인 리스트
article_file = open('/Users/hayeong/DataScience/Data-Science/news_list.txt', 'r')      # 크롤링한 기사 제목들의 텍스트 파일 (구분자는 '\n'으로 가정)

# articles_dict: {name: <연예인 이름 리스트>, article: <기사 제목 리스트>}
articles_dict = {'Name':[], 'Article':[]}

star_df.head(10)

,ID,Name,Category,Age
0,2470,BMK,가수,47
1,2469,JK김동욱,가수,44
2,2468,KAI,가수,26
3,2467,MC딩동,배우,41
4,2466,MC몽,가수,41
5,2465,RM,가수,26
6,2464,T.O.P,가수,32
7,2463,가인,가수,33
8,2462,가희,배우,39
9,2461,간미연,가수,38


# articles_dict 생성 후 데이터프레임 만들기
: 기사 제목에 특정 연예인 이름이 들어있으면 'Article', 'Name' 키에 각각 저장 후 article_df 생성

* article: 기사 제목
* star_df: celeb_name_list.csv의 데이터프레임. star_df['Name']에 연예인 이름(=name)이 저장되어있음
* article_df: articles_dict의 데이터프레임. columns=['Name', 'Article'] 


In [9]:
while True:
    # 크롤링한 기사 제목 한 줄을 읽어옴
    article = article_file.readline()
    # 크롤링한 기사 제목을 모두 읽었으면 break
    if article == '':
        break
    #article = article[:-1]       # 기사 제목 뒤의 \n 제외
    # namelist 안의 모든 연예인 이름에 대해서 기사 제목에 포함되는 이름이 있는 지 확인
    for name in star_df['Name']:
        # 보통 연예인 이름 뒤에 컴마가 오길래 '<이름>,'를 찾는걸로 해봄
        if name+',' in article or name+' ' in article or name+"'" in article:
            # 이름으로 쓰인 단어가 아닐 때는 제외함 
            if article.find(name) != 0 and article[article.find(name)-1] != ' ':
                continue
            
            if not (name in articles_dict['Name']):
                articles_dict['Name'].append(name)
                articles_dict['Article'].append(article)
            else:
                index = articles_dict['Name'].index(name)
                if not (article in articles_dict['Article'][index]):
                    articles_dict['Article'][index] += article
                
        # 제목에 포함된 연예인을 찾았어도 연예인 리스트 전체 탐색함
        # => 기사에 두 명 이상의 연예인이 있을 경우 모두 찾음
article_df = pd.DataFrame(articles_dict)

길>>이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는 건강미

지우>>'48.6kg' 김지우, ♥레이먼킴도 놀랄 건강미 가득 보디라인

여진>>‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신 [SNS★컷]

진>>‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신 [SNS★컷]

진>>신현빈, 건강미X고혹미 다 가진 매력부자 '심쿵주의보' [화보]

현빈>>신현빈, 건강미X고혹미 다 가진 매력부자 '심쿵주의보' [화보]

시원>>류시원, 재혼 후 근황…49세 나이 믿기지 않는 동안+건강미

엘>>에바 포비엘, 건강미 넘치는 브라톱+레깅스 자태..군살이 하나도 없네

견미리>>'견미리 딸' 이유비, 구릿빛 피부 공개…건강미 넘쳐

비>>'견미리 딸' 이유비, 구릿빛 피부 공개…건강미 넘쳐

뷔>>'10월 가수 데뷔' 김해리, 대체불가 건강미 여신 '완벽 바디라인' [화보]

정민>>조정민, 선글라스 광고 모델 발탁…넘치는 볼륨감으로 남다른 건강미 과시

비>>이유비 화보, 건강미+시크한 매력으로 젠더리스 스타일까지 완벽 소화

선미>>'이병헌 동생' 이지안, 건강미 넘치는 각선미 "볼빨간 사십대"

이병헌>>'이병헌 동생' 이지안, 건강미 넘치는 각선미 "볼빨간 사십대"

최은경>>"48세 맞아?"…최은경, 건강미 넘치는 바디프로필 공개

수현>>[MD포토] 치어리더 배수현, '건강미 가득한 깜찍함'

진>>'김사랑 몸 대역' 정현우 프로, 건강미 넘치는 수영복 화보...골프로 다져진 ...

진>>박결 '운동으로 다져진 건강미'

진>>공효진, 화보 통해 여전한 건강미 뽐내

정민>>조정민, 건강미 넘치는 수영복 자태 '화보 같은 일상'

진>>'김사랑 몸 대역' 정현우 프로, 건강미 넘치는 수영복 화보...골프로 다져진 ...

진>>박결 '운동으로 다져진 건강미'

유이>>“건강미란 이런 것”...유이, 하의실종 파격 화보 공개 [EN스타]

진>>공효진, 화보 통해 여전한 건강미 뽐내

진>>유이, 아찔+과감한 화보 통해 폴댄스로 다져진 건강미 발산

정민

비>>김희정, '힙'한 다이어트 티저 영상 공개…건강미 넘치는 '워너비 몸매'

지민>>[MD포토] 모델 원지민, '건강미 가득'

길>>배럴, 배럴데이 '최대 50％ 할인'..건강미 돋보이는 고준희 화보 '눈길'

은지>>'건강미 뿜뿜' 정은지 "운동돌이라 불러주세요" [화보]

은지>>레깅스+탱크톱..정은지, 섹시한 건강미

정은지>>레깅스+탱크톱..정은지, 섹시한 건강미

한경>>[BC카드·한경 레이디스컵 2020] 유현주, '건강미 넘치는 모습으로'

길>>이하늬, 건강미 넘치는 수영복 자태 '눈길'

선미>>[MD포토] 안희연, '건강미 넘치는 각선미'

키>>[bnt포토] 위키미키 김도연 '순백의 건강미'

은지>>정은지, 딱 봐도 탄탄한 팔 각선미...운동으로 완성한 건강미

수현>>[포토] 치어리더 배수현 '건강미 물씬'

수현>>[포토] 치어리더 배수현, 건강미 넘치는 구리빛

수지>>신수지 '역대급 건강미' 화보컷 '스포티+관능美' 폭발

비>>[MD포토] 류세비 '당당한 건강미'

김종국>>[포토]김종국, 건강미 넘쳐

김종국>>[포토]김종국, 건강미 넘쳐

김종국>>[포토]김종국, 건강미 넘쳐

김종국>>[포토]김종국, 건강미 넘쳐

길>>AOA 설현, 청순·건강미 넘치는 뷰티화보 공개 '눈길'

진>>이시영, 운동으로 다진 탄탄 몸매..기럭지+건강미 독보적

윤주>>"출산 3개월만의 복귀"…장윤주, 명불허전 건강미

엘>>배우 이엘, 피트니스 화보 ‘넘치는 건강미’

선미>>마마무 휘인, 건강미 넘치는 각선미 [2018 코리아 뮤직 페스티벌]



In [10]:
article_df.head(10)

,Name,Article
0,김혜수,"김혜수, 건강미 넘치는 일상 '시선강탈'\n"
1,김성령,"김성령, 건강미 넘치는 프리다이빙…""54세 몸매 맞아?""\n김성령 “물구나무서기 너..."
2,이하늬,"이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는 건강미\n이하늬, 싱그러운 건강미..."
3,김희정,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n김희정, 건강미 넘치는 화보 ‘치..."
4,최여진,"‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신 [SNS★컷]\n최여진, 서핑보드 ..."
5,류화영,"류화영, 건강미 돋보이는 테니스장에서의 하루\n류화영 ""#건강미 #섹시한 #종아리알..."
6,신현빈,"신현빈, 건강미X고혹미 다 가진 매력부자 '심쿵주의보' [화보]\n"
7,류시원,"류시원, 재혼 후 근황…49세 나이 믿기지 않는 동안+건강미\n"
8,유이,"유이, 피트니스 콘텐츠 구독자 수 10만 돌파..건강미 넘치는 행보\n‘나 혼자 산..."
9,이유비,"'견미리 딸' 이유비, 구릿빛 피부 공개…건강미 넘쳐\n이유비 화보, 건강미+시크한..."


# count_dict 생성 후 데이터프레임 만들기
: 각각의 연예인 이름이 몇 번 언급되어있는지를 저장함
'Name'키에 연예인 이름, 'Count'키에 언급 횟수를 저장 후 count_df 생성


In [11]:
# count_dict: {Name : 한 번 이상 언급된 연예인 이름 리스트, Count : 언급 횟수}
count_dict = {'Name': articles_dict['Name'], 'Count': []}
i = 0
for name in articles_dict['Name']:
    count_dict['Count'].append(articles_dict['Article'][i].count('\n'))
    i +=1

count_df = pd.DataFrame(count_dict)
count_df.head()

,Name,Count
0,김혜수,1
1,김성령,2
2,이하늬,16
3,김희정,16
4,최여진,3


# 데이터프레임 전처리

1. 연예인 리스트 데이터프레임(star_df)과 연예인별 언급횟수 데이터프레임(count_df) 합침
2. 언급 되지 않은 연예인이 있는 행은 삭제 (count 컬럼이 NaN인 경우 해당 행 삭제)
3. count 컬럼을 기준으로 내림차순 정렬

In [12]:
# df에 언급 횟수인 Count 컬럼 추가
merged_df = pd.merge(star_df, count_df, on='Name', how = 'outer')
merged_df.head()

,ID,Name,Category,Age,Count
0,2470,BMK,가수,47,NaN
1,2469,JK김동욱,가수,44,NaN
2,2468,KAI,가수,26,NaN
3,2467,MC딩동,배우,41,NaN
4,2466,MC몽,가수,41,NaN


In [13]:
# 언급 되지 않은 연예인이 있는 행은 삭제
temp_df = merged_df.dropna(axis = 0, how = 'any')
# 언급 횟수 기준 내림차순으로 정렬
sorted_df = temp_df.sort_values(by = 'Count', ascending=False,)

# final_df columns: 연예인 정보 + 언급 횟수 + 언급된 기사 제목
final_df = pd.merge(sorted_df, article_df, on='Name', how = 'outer')
final_df.head(20)

,ID,Name,Category,Age,Count,Article
0,1127,유이,배우,32,29.0,"유이, 피트니스 콘텐츠 구독자 수 10만 돌파..건강미 넘치는 행보\n‘나 혼자 산..."
1,1477,설현,가수,25,20.0,"AOA 설현, 농심 '켈로그' 모델 발탁…""건강미 대표 이미지 부합""\nAOA 설현..."
2,1460,소유,가수,28,18.0,"소유, 신곡 '가라고' 안무 영상 공개…탄탄한 건강미\n소유, 나이 29살 싱글녀의..."
3,882,이시영,배우,38,18.0,"이시영, 흑백 뚫고 나오는 걸크러시…만능 운동러의 건강미\n이시영, 건강미 넘치는 ..."
4,746,이하늬,배우,37,16.0,"이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는 건강미\n이하늬, 싱그러운 건강미..."
5,1922,김희정,배우,28,16.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n김희정, 건강미 넘치는 화보 ‘치..."
6,1923,김희정,배우,49,16.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n김희정, 건강미 넘치는 화보 ‘치..."
7,1487,선미,가수,28,12.0,"선미 '보라빛 밤' 티저 공개, 스윔수트 입고 뽐낸 건강미\n‘8kg 증량’ 선미,..."
8,2039,김종국,가수,44,11.0,"[포토] 김종국, 한결같은 건강미\n[포토] 김종국, 넘치는 건강미\n[bnt포토]..."
9,2038,김종국,개그맨,57,11.0,"[포토] 김종국, 한결같은 건강미\n[포토] 김종국, 넘치는 건강미\n[bnt포토]..."


# 데이터 확인하기

1. 동명이인인 데이터 확인
2. 상위 20명의 후보 연예인 리스트 확인
3. 후보 연예인들이 언급되었던 기사가 제대로 된 것인지 확인
4. 최종 데이터프레임을 csv 파일로 저장

In [14]:
### 생략 가능 ###
# 동명이인일 시 두 명 다 같은 언급횟수를 가지고 있음  
# ex)김희정(49세), 김희정(28세): 기사 하나에 언급되면 둘 다 count = 1이 됨
# duplicated: 동명이인인 이름의 리스트
duplicated = final_df[final_df.Name.duplicated()]['Name'][:20].tolist()
print('=== 동명이인인 이름 리스트 ===')
print(duplicated)


# 동명이인이 들어간 기사 체크하기
for key in duplicated:
    print('>> '+key+'\n' + articles_dict['Article'][articles_dict['Name'].index(key)])

# 동명이인의 정보 확인하기
duplicated_df = final_df[final_df['Name'].isin(duplicated)]
duplicated_df.head(100)

=== 동명이인인 이름 리스트 ===
['김희정', '김종국', '현아', '한혜진', '지성', '김지훈', '김지훈']
>> 김희정
김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄
김희정, 건강미 넘치는 화보 ‘치명적인 매력’
김희정, 군살 제로 건강미..튼튼 꿀벅지 '깜짝'[★SHOT!]
김희정, 독보적 건강미 발산…다이어트도 힙하게
김희정, 다이어트 CF로 건강미 과시
김희정, 구릿빛 피부에 빛나는 몸매…건강미 넘치네
김희정, 건강미에 더한 몽환적 분위기…치명적인 매력 [화보]
김희정, 화보 비하인드컷 공개...'러블리+건강미'
김희정, 건강미 넘치는 '워너비 몸매' 담은 티저 영상
김희정, '건강미+몽환적 분위기' 발산하는 유니크한 매력 [화보]
김희정, 구릿빛 피부+탄탄 몸매로 '섹시 건강미' 뿜뿜
김희정, 비키니·트레이닝복까지 완벽 소화…건강미 넘치는 매력 [화보]
김희정, 준PO 4차전 시구..복근 드러낸 '건강미'
배우 김희정, SNS에 건강미 넘치는 수영복 차림 공개
김희정, '힙'한 다이어트 티저 영상 공개…건강미 넘치는 '워너비 몸매'
김희정, 건강미 넘치는 구릿빛 피부

>> 김종국
[포토] 김종국, 한결같은 건강미
[포토] 김종국, 넘치는 건강미
[bnt포토] 김종국 '건강미를 챙겨요'
[bnt포토] 김종국 '흘러 넘치는 건강미'
[포토] 김종국, '넘치는 건강미'
김종국, 흘러넘치는 건강미 '퍼펙트 하죠?' [MD동영상]
[포토] 김종국, 건강미 가득
[포토] 김종국, 건강미 철철~
[머니S포토] 김종국 '건강미 넘치는 미소'
'여자 김종국' 배그린, 꿀복근 공개…탄탄한 건강미
[포토Q] 김종국 '흘러넘치는 건강미'

>> 현아
‘원더풀’ 현아, 美친 건강미 [화보]
현아, 화보에서 美친 건강미 과시
현아, 건강미 넘치는 스포티+섹시美 [화보]
현아, 美친 보디라인 자랑…넘사벽 건강미에 감탄이 절로 [화보]
현아, 속옷 화보서 탄탄 바디라인 과시…'건강미 폭발'
현아, 스포티한 레깅스 패션…건강미 '물씬'
[포토] 현아, 건강미 발산

,ID,Name,Category,Age,Count,Article
5,1922,김희정,배우,28,16.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n김희정, 건강미 넘치는 화보 ‘치..."
6,1923,김희정,배우,49,16.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n김희정, 건강미 넘치는 화보 ‘치..."
8,2039,김종국,가수,44,11.0,"[포토] 김종국, 한결같은 건강미\n[포토] 김종국, 넘치는 건강미\n[bnt포토]..."
9,2038,김종국,개그맨,57,11.0,"[포토] 김종국, 한결같은 건강미\n[포토] 김종국, 넘치는 건강미\n[bnt포토]..."
13,74,현아,가수,28,7.0,"‘원더풀’ 현아, 美친 건강미 [화보]\n현아, 화보에서 美친 건강미 과시\n현아,..."
14,75,현아,가수,33,7.0,"‘원더풀’ 현아, 美친 건강미 [화보]\n현아, 화보에서 美친 건강미 과시\n현아,..."
21,110,한혜진,배우,38,5.0,"[포토] 한혜진, 건강미 넘치는 미모\n'나 혼자 산다' 한혜진, '섹시에 건강미 ..."
22,109,한혜진,모델,37,5.0,"[포토] 한혜진, 건강미 넘치는 미모\n'나 혼자 산다' 한혜진, '섹시에 건강미 ..."
32,376,지성,배우,43,3.0,걸그룹 소녀주의보 지성 “60kg아이돌 안녕”···건강미 넘치는 근황\n‘60kg ...
33,375,지성,가수,18,3.0,걸그룹 소녀주의보 지성 “60kg아이돌 안녕”···건강미 넘치는 근황\n‘60kg ...


In [15]:
candidates = final_df['Name'].tolist()[:20]
print('=== Top 20 ===')
print(candidates)

=== Top 20 ===
['유이', '설현', '소유', '이시영', '이하늬', '김희정', '김희정', '선미', '김종국', '김종국', '효린', '조여정', '클라라', '현아', '현아', '제시', '이청아', '한예슬', '김재경', '루나']


In [16]:
# 상위 20명의 기사가 제대로 된 것인지 체크하기
print('=== Check articles ===')
for key in candidates:
    print('>> '+key+'\n' + articles_dict['Article'][articles_dict['Name'].index(key)])

=== Check articles ===
>> 유이
유이, 피트니스 콘텐츠 구독자 수 10만 돌파..건강미 넘치는 행보
‘나 혼자 산다’ 유이, 건강미 폭발하는 폴댄스 공개
유이, 티셔츠+레깅스 조합이 이렇게 잘 어울리다니…‘건강미’ 넘치는 근황
유이, 탄탄한 바디라인 드러내는 건강미 넘치는 화보 공개
[오마이포토] 'SF8' 유이, 건강미 넘치는 미소
유이, 운동으로 완성된 완벽 S라인 '건강미 발산'
[화보IS] 유이, 운동으로 완성한 탄탄한 건강미
[화보] 유이, 파격 세미 누드로 뽐낸 건강미
배우 유이, 건강미 넘치는 행보 눈길
유이, 파격 세미누드 화보 공개..'꿀벅지' 건강미 되찾았다 [화보]
유이, 아찔+과감한 화보 통해 폴댄스로 다져진 건강미 발산
유이, 아름다운 보디라인+건강미...세미누드 화보 공개
유이, 완벽하게 되찾은 건강미..인형 같은 제주 소녀 [화보]
'나혼산' 유이, 건강미 폭발 파워 폴댄스→숨겨온 솔직한 속내까지 [Oh!쎈 예...
유이, 몸매 이정도였어? 탄탄한 건강미
유이, 되찾은 건강미…리즈는 지금 [화보]
유이, 핫보디 아이콘의 아름다운 건강미(화보)
유이, 숲속에서 우아한 화보 ‘건강미 넘치네’
유이 ‘유이의 라스트핏’ 메인 MC 발탁 ‘건강미 넘치는 행보’
유이, 바다 앞 여유로운 미소 ‘건강미’ 뿜뿜 [SNS★컷]
유이, 거식증 루머 심경고백→'파격' 세미누드 공개→건강美 발산[종합]
유이, 과감한 세미 누드…아름다운 건강美 [화보]
'나 혼자 산다' 유이, 폴댄스로 탄탄한 몸매+건강미 과시
유이, 요가로 다져진 건강미…"나를 사랑해주는 것이 건강한 삶"
유이, 꿀벅지 레전드 다운 핫 바디 공개...건강미 넘치는 싱글즈 7월 호 커버
유이 파격 화보, 아찔 세미누드 공개…청순 미모+절제된 건강미 발산
'나 혼자 산다' 유이, 건강미 폭발하는 풀 파워 폴 댄스! 파워는 완성! 유연성...
유이, 깡마른 몸매에도 살아있는 '건강미'[★SHOT!]
유이, 몸매 고민·상처 날려버린 세미누드

>> 설현
AOA 설현, 농심 '

In [17]:
import qgrid
qgrid_widget = qgrid.show_grid(final_df,show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [18]:
# csv로 저장
final_df.to_csv('candidates_final.csv', index=False, mode='w', encoding='utf-8-sig')